In [41]:
# hide
from typing import List, Dict, Tuple

# Algorithms to know
> "Common algorithms to know"
- toc: true
- branch: master
- hide: false
- search_exclude: true
- show_tags: false

## Quicksort

**Purpose**: Sort array

**Idea**:
> Recursive sorting of smaller arrays until array size is 1 and there's nothing to sort.
Pick a pivot, create array of smaller elements, and array of larger elements
run sorting on smaller arrays. if array size is 1 return the array

**Time Complexity**: O(nlog(n)), log(n) levels due to splitting the arrays, for each levels
we go through the whole array.

### Code

In [30]:
def quicksort(array: List[int]) -> List[int]:
    """ Sort array of numbers from smallest to largest"""
    # Base case of recursion
    # array of size, nothing to sort
    if len(array) < 2:
        return array

    # Pick pivot at the middle of the array
    pivot = array[len(array) // 2]

    # create array of smaller than pivot elements
    smaller_arr = [el for el in array if el < pivot]

    # create array of larger than pivot elements
    larger_arr = [el for el in array if el > pivot]

    # run sorting on the array of smaller and larger elements
    # and combine with the pivot element in the middle
    return quicksort(smaller_arr) + [pivot] + quicksort(larger_arr)

### Example

In [31]:
import random
# sample 10 random numbers between 1 to 100
array = random.sample(range(1, 100), 10)

print(f"Input array: {array}")
print(f"Sorted array: {quicksort(array)}")

Input array: [25, 42, 27, 95, 47, 13, 94, 96, 4, 99]
Sorted array: [4, 13, 25, 27, 42, 47, 94, 95, 96, 99]


## Binary search

**Purpose**: Find element in a sorted array.

**Idea**:
>Compare element to middle element in the array
Based on comparison reduce search to half the array
continue until element is found or no elements remain in array

**Time complexity**: $O(\log n)$

### Code

In [32]:
def binarysearch(array: List[int], number: int) -> int:
    """
    Find element in a sorted array
    """

    # initialize start and end indices
    start = 0
    end = len(array)

    while start < end:
        middle = (start + end) // 2  # get middle index of array
        if number == array[middle]:  # element was found
            return middle
        if array[middle] < number:
            # middle element is smaller than number, update start to after middle index
            # keep end the same. This halves the search array to the right
            start = middle + 1
        else:
            # middle element is larger than number, update end to before middle index
            # keeps start the same. This halves the search array to the left.
            end = middle - 1
    # loop has finished with an empty array, number not found
    return -1

### Example

In [33]:
# sample 100 random numbers between 1 to 1000, and sort
array = sorted(random.sample(range(1, 100), 10))
print(f"Input array: {array}")

# sample random number from array
number = random.sample(array, 1)[0]
print(f"Number to find: {number}")

print(f"Index found: {binarysearch(array, number)}")

Input array: [4, 18, 20, 21, 30, 34, 43, 47, 74, 96]
Number to find: 47
Index found: -1


## Graph search

**Purpose**:
Breadth-First (BF) search is used to find if a path to a node exists in a graph
and finds the shortest path.

**Idea**:
>Go through all neighbors of a node before moving to the next-neighbor nodes.
Keep neighbors in a queue to go through them by order of insertion.

**Time Complexity**: $O(E+V)$, at most going through every edge ($E$) and adding each edge to a queue ($V$)

### Code

In [34]:
from collections import deque

def trace_path(parents, start, end):
    """ Trace path back from end to start using parents """
    path = [end]
    while path[-1] != start:
        path.append(parents[path[-1]])
    path.reverse()
    return path


def graphsearch(graph: Dict[str, List[str]], start: str, end: str) -> bool:
    """
    Search shortest path from start to end node in a graph using breadth-first search
    """

    queue = deque()  # create empty queue
    queue += [start]  # add start node to queue
    searched: List[str] = []  # initialise list of processed nodes
    # initialise dictionary fo parent of node for tracing back path
    parents: Dict[str, str] = {}

    while queue:
        node = queue.popleft()  # get node from queue
        if node not in searched:  # process node if not already processed
            if node == end:  # found end node
                print(f"path to {end} found")
                print(f"path: {trace_path(parents, start, end)}")
                return True
            else:
                queue += graph[node]  # add neighbors of node to the queue
                searched.append(node)  # add node to processed nodes to not repeat
                for n in graph[node]:  # add parent for each of node's neighbors
                    parents[n] = node

    # queue is empty, path not found
    print(f"path to {end} not found")
    return False

### Example

<img src='../assets/simple_graph.png' width='200px'>

In [35]:
graph = {}
graph["Mark"] = ["Alice", "Bob"]
graph["Alice"] = ["David"]
graph["Bob"] = ["John"]
graph["John"] = ["David"]
graph["David"] = []
graph["Alex"] = ['Sven']
graph["Sven"] = []

found = graphsearch(graph, "Mark", "David")
print()
found = graphsearch(graph, "Mark", "John")
print()
found = graphsearch(graph, "Mark", "Sven")

path to David found
path: ['Mark', 'Alice', 'David']

path to John found
path: ['Mark', 'Bob', 'John']

path to Sven not found


## Dijkstra algorithm

**Purpose**: Finds shortest path (weight-wise) to a node in a graph.

**Idea**:
> Find smallest weight node using a min-heap data structure,
there is no cheaper way to get to that node.
Check if there's a cheaper way to get to its neighbors, update distances, min-heap
and parents. Repeat for all nodes in the graph, keeping track of distances to nodes.


**Complexity**: $O(E\log(V))$, the log comes from the min-heap
alternative implementation with an array instead of min-heap will have $O(V^2)$


### Code

In [43]:
import heapq  # standard library min-heap

def trace_path(parents, start, end):
    """ Trace path back from end to start using parents """
    path = [end]
    while path[-1] != start:
        path.append(parents[path[-1]])
    path.reverse()
    return path


def dijkstra(graph: Dict[str, Dict[str, float]], start: str, end: str) -> Tuple[List[str], float]:
    """ 
    Find shortest weighted path between nodes in a non-negative weights
    directed and acyclic graph 
    """

    # Initialize dictionary of distances and parents
    distances: Dict[str, float] = {}
    parents: Dict[str, str] = {}

    # Initialize a min-heap of weights
    weights: List = []
    heapq.heapify(weights)

    # Set all distances to nodes to infinity
    infinity = float("inf")
    for node in graph:
        parents[node] = ""
        distances[node] = infinity

    # add start node to min-heap with weight zero, and set distance to zero
    heapq.heappush(weights, (0, start))
    distances[start] = 0

    # Get lowest weight nodes to start going through its neighbors
    while weights:
        node = heapq.heappop(weights)  # get node from min-heap

        neighbors = graph[node[1]]  # current node's neighbors
        for n in neighbors.keys():  # loop over neighbors
            new_dist = distances[node[1]] + neighbors[n]
            if new_dist < distances[n]:
                # update the weights and parent only if the new weighted path weighs less
                # in that case a more efficient path to the neighbor has been found
                distances[n] = new_dist
                parents[n] = node[1]
                heapq.heappush(weights, (new_dist, n))

    # get path of smallest weight using parents dictionary
    path = trace_path(parents, start, end)
    # get final distance of optimal path
    distance = distances[end]
    print(
        f"Shortest weighted-path from {start} to {end}: {path}, with total distance: {distance}"
    )
    return path, distance

### Example

<img src='../assets/weighted_graph.png' width='200px'>

In [42]:
# define graph
graph: Dict[str, Dict[str, float]] = {}

graph["A"] = {}
graph["A"]["B"] = 2
graph["A"]["C"] = 5
graph["B"] = {}
graph["B"]["C"] = 1
graph["B"]["D"] = 6
graph["C"] = {}
graph["C"]["D"] = 2
graph["D"] = {}

# find path and weight
path, weight = dijkstra(graph, "A", "D")
path, weight = dijkstra(graph, "A", "C")

Shortest weighted-path from A to D: ['A', 'B', 'C', 'D'], with total distance: 5
Shortest weighted-path from A to C: ['A', 'B', 'C'], with total distance: 3
